In [13]:
import math

class ResistorPicker(object):
    """Picks 1% resistor values for a desired i2c address given a minimum resistance constraint.
    
    Breakdown of Constraints satisfied:
    -- resistors have a minimum resistance
    -- resistors fall within +/-0.015 of the setpoint XORL or XORH ratios
    -- resistors real 1% resistor values
    """
    X_RATIO_TOLERANCE = 0.015
    OHM_STEP = 1000
    
    bits_to_xorl_over_vcc = {0b0000: 0.0,
                    0b0001: .09375,
                    0b0010: .15625,
                    0b0011: .21875,
                    0b0100: .28125,
                    0b0101: .34375,
                    0b0110: .40625,
                    0b0111: .46875,
                    0b1000: .53125,
                    0b1001: .59375,
                    0b1010: .65625,
                    0b1011: .71875,
                    0b1100: .78125,
                    0b1101: .84375,
                    0b1110: .90625,
                    0b1111: .96875}
    bits_to_xorh_over_vcc = {0b000: 0.0,
                    0b001: .09375,
                    0b010: .15625,
                    0b011: .21875,
                    0b100: .28125,
                    0b101: .34375,
                    0b110: .40625,
                    0b111: .46875}
    
    # significant-figure equation from:
    # https://stackoverflow.com/questions/3410976/how-to-round-a-number-to-significant-figures-in-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa    
    R_E96_10K = [10**(i/96) * 10000 for i in range(0, 96)]
    R_E96_10K = [round( x, int( 3 - math.ceil(math.log10(x)) ) ) for x in R_E96_10K]
    
    R_E96_1K = [10**(i/96) * 1000 for i in range(0, 96)]
    R_E96_1K = [round( x, int( 3 - math.ceil(math.log10(x)) ) ) for x in R_E96_1K]
    
    def __init__(self, min_total_r, default_device_address):
        self.r_total = min_total_r
        self.default_device_address = default_device_address
        return
    
    def compute_resistor_values(self, desired_address):
        #print("Desired Address: {}".format(hex(desired_address)))
        des_xorl_ratio, des_xorh_ratio = self.get_xorl_and_xorh_ratios(desired_address)
        #print("desired xorl_ratio: {} | desired xorh_ratio: {}".format(des_xorl_ratio, des_xorh_ratio))
        ra1, ra2, ra3 = self.get_resistor_triad(des_xorl_ratio, des_xorh_ratio)
        actual_xorl_ratio, actual_xorh_ratio = self._get_x_ratios_from_resistors(ra1, ra2, ra3)
        
        #print("actual xorl_ratio:  {:.5f} | actual xorh_ratio:  {:.5f}".format(actual_xorl_ratio, actual_xorh_ratio))
        if not (self.x_ratio_in_bounds(des_xorl_ratio, actual_xorl_ratio) and \
                self.x_ratio_in_bounds(des_xorh_ratio, actual_xorh_ratio)):
            old_r_total = self.r_total
            self.r_total += ResistorPicker.OHM_STEP
            print("failed to find real r-triad {} for address {} and min total r: {}"
                  .format((ra1, ra2, ra3), hex(desired_address), self.r_total))
            return self.compute_resistor_values(desired_address)
            self.r_total = old_r_total
            return
        return (ra1, ra2, ra3)
    
    def get_resistor_triad(self, xorl_over_vcc, xorh_over_vcc):
        r_a3 = self.r_total * xorh_over_vcc
        r_a2 = self.r_total * xorl_over_vcc - r_a3
        r_a1 = self.r_total - r_a3 - r_a2
        #print("{} | {} | {}".format(r_a1, r_a2, r_a3))
        if r_a3 <= 9760:
            r_a3 = ResistorPicker._r_to_e961k(r_a3)
        else:
            r_a3 = ResistorPicker._r_to_e9610k(r_a3)
        if r_a2 <= 9760:
            r_a2 = ResistorPicker._r_to_e961k(r_a2)
        else:
            r_a2 = ResistorPicker._r_to_e9610k(r_a2)
        if r_a1 <= 9760:
            r_a1 = ResistorPicker._r_to_e961k(r_a1)
        else:
            r_a1 = ResistorPicker._r_to_e9610k(r_a1)

        return (r_a1, r_a2, r_a3)
    
    @classmethod
    def _r_to_e9610k(cls, resistor_value):
        """Compute the closest 10K-100K 1% resistor value for the given resistance value.
        """
        if resistor_value == 0:
            return 0
        index = cls._r_to_e9610k_index(resistor_value)
        return cls.R_E96_10K[index]
    
    @staticmethod
    def _r_to_e9610k_index(resistor_value):
        """Compute the index to the closest 10K-100K 1% resistor value for the given resistance value.
        This is just the inverse function for generating the E96 resistor values.
        """
        index = round(math.log((resistor_value/10000)**(96/math.log(10))))
        if 0 > index > 95:
            raise IndexError("{} out of bounds.".format(index))
        return index
    
    @classmethod
    def _r_to_e961k(cls, resistor_value):
        """Compute the closest 1K-10K 1% resistor value for the given resistance value.
        """
        if resistor_value <= 10:
            return 0
        index = cls._r_to_e961k_index(resistor_value)
        return cls.R_E96_1K[index]
    
    @staticmethod
    def _r_to_e961k_index(resistor_value):
        """Compute the index to the closest 1K-10K 1% resistor value for the given resistance value.
        This is just the inverse function for generating the E96 resistor values.
        """
        index = round(math.log((resistor_value/1000)**(96/math.log(10))))
        if 0 > index > 95:
            raise IndexError("{} out of bounds.".format(index))
        return index
    
    def _get_translation(self, desired_address):
        translation = self.default_device_address ^ desired_address
        return translation
        
    def get_xorl_and_xorh_ratios(self, desired_address):
        translation_byte = self._get_translation(desired_address)
        upper_bits = (translation_byte & 0x70) >> 4
        lower_bits = (translation_byte & 0x0F)
        xorl_ratio = ResistorPicker.bits_to_xorl_over_vcc[lower_bits]
        xorh_ratio = ResistorPicker.bits_to_xorh_over_vcc[upper_bits]
        return [xorl_ratio, xorh_ratio]
    
    def _get_x_ratios_from_resistors(self, ra1, ra2, ra3):
        r_total = ra1 + ra2 + ra3
        xorh_ratio = ra3/r_total
        xorl_ratio = (ra2 + ra3)/r_total
        return [xorl_ratio, xorh_ratio]
    
    @classmethod
    def x_ratio_in_bounds(cls, desired_x_ratio, actual_x_ratio):
        return abs(desired_x_ratio - actual_x_ratio) <= cls.X_RATIO_TOLERANCE

In [18]:
address_range = range(0x20, 0x38)
picker = ResistorPicker(90000, 0x28)
address_booklet = {}
for address in address_range:
    ra123 = picker.compute_resistor_values(address)
    #print("ra1 = {} | ra2 = {} | ra3 = {}".format(ra1, ra2, ra3))
    address_booklet[hex(address)] = ra123
import pprint
pprint.pprint(address_booklet)

{'0x20': (42200.0, 47500.0, 0),
 '0x21': (36500.0, 53600.0, 0),
 '0x22': (30900.0, 59000.0, 0),
 '0x23': (25500.0, 64900.0, 0),
 '0x24': (19600.0, 69800.0, 0),
 '0x25': (14000.0, 76800.0, 0),
 '0x26': (8450.0, 82500.0, 0),
 '0x27': (2800.0, 86600.0, 0),
 '0x28': (90900.0, 0, 0),
 '0x29': (82500.0, 8450.0, 0),
 '0x2a': (76800.0, 14000.0, 0),
 '0x2b': (69800.0, 19600.0, 0),
 '0x2c': (64900.0, 25500.0, 0),
 '0x2d': (59000.0, 30900.0, 0),
 '0x2e': (53600.0, 36500.0, 0),
 '0x2f': (47500.0, 42200.0, 0),
 '0x30': (42200.0, 39200.0, 8450.0),
 '0x31': (36500.0, 45300.0, 8450.0),
 '0x32': (30900.0, 51100.0, 8450.0),
 '0x33': (25500.0, 56200.0, 8450.0),
 '0x34': (19600.0, 61900.0, 8450.0),
 '0x35': (14000.0, 68100.0, 8450.0),
 '0x36': (8450.0, 73200.0, 8450.0),
 '0x37': (2800.0, 78700.0, 8450.0)}
